## Handling depth in FESOM

Variables in FESOM located at different depths. Most of them associated to the middle depth of the layer (69 vertical levels), but some at layer boundaries (70 vertical levels). 

For the data presented on hackathon, the only variable that located on 70 levels is avg_wo.

However healpix output always have 70 vertical levels, and have no depth associated with those levels. Here we define two vertical coordinates:
`nz1` - the one you most probably need (mid levels depths)
`nz` - only use if you plan to use `avg_wo`

In the HEALPix output of variables defined on `nz1` the last level contains the same values as the second to last one, and we repeat the vertical coordinate value in `nz1` coordinate (`6175`). 

In [35]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.feature as cfeature
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import gribscan
import intake
import eccodes
import healpy as hp
import os

In [40]:
def attach_depth_coords(ds):
    nz1  = [2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5, 57.5,
    62.5, 67.5, 72.5, 77.5, 82.5, 87.5, 92.5, 97.5, 105, 115, 125, 135, 145,
    155, 165, 175, 185, 195, 210, 230, 250, 270, 290, 320, 360, 400, 440,
    480, 520, 560, 600, 640, 710, 810, 950, 1110, 1255, 1415, 1600, 1810,
    2035, 2275, 2525, 2775, 3025, 3275, 3525, 3775, 4025, 4275, 4525, 4775,
    5025, 5275, 5525, 5825, 6175, 6175]
    nz = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
    90, 95, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 220, 240,
    260, 280, 300, 340, 380, 420, 460, 500, 540, 580, 620, 660, 760, 860,
    1040, 1180, 1330, 1500, 1700, 1920, 2150, 2400, 2650, 2900, 3150, 3400,
    3650, 3900, 4150, 4400, 4650, 4900, 5150, 5400, 5650, 6000, 6350]
    return ds.assign_coords(
        nz1 = (("level",), nz1, {"units": "m"}),
        nz = (("level",), nz, {"units": "m"}),
    )

In [38]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")

In [41]:
ds = cat.IFS['IFS_9-FESOM_5-production']['3D_daily_healpix512_ocean'].to_dask().pipe(attach_depth_coords)

In [42]:
ds

<xarray.Dataset>
Dimensions:     (time: 8990, level: 70, value: 3145728)
Coordinates:
    lat         (value) float64 dask.array<chunksize=(3145728,), meta=np.ndarray>
  * level       (level) int64 1 2 3 4 5 6 7 8 9 ... 62 63 64 65 66 67 68 69 70
    lon         (value) float64 dask.array<chunksize=(3145728,), meta=np.ndarray>
  * time        (time) datetime64[ns] 2020-01-21 2020-01-22 ... 2045-01-01
    nz1         (level) float64 2.5 7.5 12.5 ... 5.825e+03 6.175e+03 6.175e+03
    nz          (level) int64 0 5 10 15 20 25 ... 4900 5150 5400 5650 6000 6350
Dimensions without coordinates: value
Data variables:
    avg_so      (time, level, value) float64 dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    avg_thetao  (time, level, value) float64 dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    avg_uoe     (time, level, value) float64 dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    avg_von     (time, level, value) float64 dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    avg_wo      (time, level, value) float64 dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
Attributes:
    edition:            2
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          1003
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...